# 4. Overexpression targets

In [3]:
%run 1_Objective_function.ipynb

ASPtex ASPtex: asp__L_e <=> asp__L_p L-aspartate transport via diffusion (extracellular to periplasm)
EX_asp__L_e EX_asp__L_e: asp__L_e -->  L-Aspartate exchange
0.87699721425716
0.8769972144269869


To begin with, a FSEOF is ran to determine potential candidate reactions for overexpression. The highest the increase from row one to row 10 the highest the possitive effect over expression would have in Aspartate productivity. 

In [4]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF
fseof = FSEOF(model)
fseof.run(target=model.reactions.EX_asp__L_e)

,1,2,3,4,5,6,7,8,9,10
PPM,-0.800024,-0.781507,-0.762991,-0.744474,-0.725958,-0.707441,-0.688925,-0.670204,-0.651416,-0.632627
ASPTA,-2.923250,-3.278808,-3.634366,-3.989924,-4.345482,-4.701040,-5.056597,-5.411516,-5.766220,-6.120924
PYK,1.961931,1.648049,1.334166,1.020284,0.706401,0.392518,0.078636,0.000000,0.000000,0.000000
PTAr,-0.499026,-0.487476,-0.475926,-0.464376,-0.452826,-0.441276,-0.429726,-0.418049,-0.406329,-0.394610
ACKr,0.499026,0.487476,0.475926,0.464376,0.452826,0.441276,0.429726,0.418049,0.406329,0.394610
ENO,15.623485,15.648072,15.672659,15.697247,15.721834,15.746421,15.771009,15.797689,15.825069,15.852448
FBA,8.411815,8.371867,8.331919,8.291970,8.252022,8.212073,8.172125,8.133559,8.095455,8.057350
PGL,2.526999,2.698633,2.870266,3.041899,3.213532,3.385166,3.556799,3.724856,3.891717,4.058579
RPE,1.019806,1.149616,1.279427,1.409237,1.539047,1.668857,1.798668,1.926263,2.053119,2.179974
TALA,0.673052,0.734181,0.795310,0.856439,0.917568,0.978697,1.039826,1.099806,1.159402,1.218998


In [5]:
from cobra.flux_analysis import flux_variability_analysis

We then selected the 3 most relevant targets for over expression (PPC, NH4tpp and GND) some of which have been reported in the literature as ideal over-expression targets to increase aspartate production. 

# Analysis of the Overexpression candidates

**PPC:** Encondes for *phosphoenolpyruvate carboxylase* and when enhanced improves the supply of oxaloacetate one of the main precursors for *L-aspartate*. It has been studied and demostreted as a viable over expression target (Piao et al., 2019). 

**Nh4tpp:** Is a ammonia trnasport reaction. Amonia and fumaric acid can be catalized to produced L-aspartate by L-aspartase and therefore an increase in amonia availabilty would result in a higher concentration of the desired product (Shibatani, 1996).

**GND:** Encondes for *phosphogluconate dehydrogenase*, which is an enzyme that partakes in pyruvate production. pyruvate is eventualy converted into fumarate, a precursor for *L-aspartate* (Shimizu, 2013). 

First we would like to determine a base line for for the fluxes going to each reaction. 

In [19]:
with model:
    overexpress = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = overexpress, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_0 = model.optimize().objective_value
    max_yield_glucose = Asp_production_0 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_0)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 


              minimum    maximum
PPC          0.000000  26.033759
NH4tpp       8.524346  21.846632
GND          0.000000  31.411827
EX_asp__L_e  0.000000   1.828549
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.5960275574887195
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.45960275574887194


With the base line stablished we then give the reactions a higher lower bound to simulate the effect of overexpressing the corresponding genes and analises the change in flux. Every reaction was tested in every possible combination of single. double and triple target overexpression. 

In [20]:
with model:
    model.reactions.PPC.lower_bound = 5
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_1 = model.optimize().objective_value
    max_yield_glucose = Asp_production_1 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_1)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

             minimum    maximum
PPC          5.00000  27.609851
NH4tpp       8.45787  21.377785
GND          0.00000  32.253746
EX_asp__L_e  0.00000   1.911583
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.596027557488512
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4596027557488512


In [21]:
with model:
    model.reactions.NH4tpp.lower_bound = 15
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_2 = model.optimize().objective_value
    max_yield_glucose = Asp_production_2 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_2)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

             minimum    maximum
PPC              0.0  23.063884
NH4tpp          15.0  26.822286
GND              0.0  31.081851
EX_asp__L_e      0.0   1.880818
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.083319780299425
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.40833197802994253


In [22]:
with model:
    model.reactions.GND.lower_bound = 5
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_3 = model.optimize().objective_value
    max_yield_glucose = Asp_production_3 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_3)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

              minimum    maximum
PPC          0.000000  24.930829
NH4tpp       8.482995  21.400519
GND          5.000000  31.977328
EX_asp__L_e  0.000000   1.845361
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.591181275892887
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4591181275892887


In [23]:
with model:
    model.reactions.PPC.lower_bound = 5
    model.reactions.NH4tpp.lower_bound = 15
    model.reactions.GND.lower_bound = 5
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_4 = model.optimize().objective_value
    max_yield_glucose = Asp_production_4 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_4)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

             minimum    maximum
PPC              5.0  24.480667
NH4tpp          15.0  26.131810
GND              5.0  32.782463
EX_asp__L_e      0.0   2.050596
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.040307520088276
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4040307520088276


In [24]:
with model:
    model.reactions.PPC.lower_bound = 5
    model.reactions.NH4tpp.lower_bound = 15
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_5 = model.optimize().objective_value
    max_yield_glucose = Asp_production_5 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_5)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

             minimum    maximum
PPC              5.0  24.712249
NH4tpp          15.0  26.264143
GND              0.0  31.576759
EX_asp__L_e      0.0   1.981854
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.083319780299401
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.40833197802994003


In [25]:
with model:
    model.reactions.NH4tpp.lower_bound = 15
    model.reactions.GND.lower_bound = 5
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_6 = model.optimize().objective_value
    max_yield_glucose = Asp_production_6 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_6)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

             minimum    maximum
PPC              0.0  22.197591
NH4tpp          15.0  26.417524
GND              5.0  31.795025
EX_asp__L_e      0.0   1.902921
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.040307520088247
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.40403075200882466


In [26]:
with model:
    model.reactions.PPC.lower_bound = 5
    model.reactions.GND.lower_bound = 5
    reactions_OE = [model.reactions.PPC, model.reactions.NH4tpp, model.reactions.GND, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = reactions_OE, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_7 = model.optimize().objective_value
    max_yield_glucose = Asp_production_7 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_7)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 

              minimum    maximum
PPC          5.000000  27.105667
NH4tpp       8.391265  21.023075
GND          5.000000  33.231785
EX_asp__L_e  0.000000   1.964948
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.591181275893015
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4591181275893015


Literature based Overexpression targets were analysed to compare against the compurer detemrined targets. The targets where takne from a study conducted by (Piao et al., 2019). They both are related to the Aspartate synthesis metabolic pathway and their overexpression is ment to maximaze the flow of glucose towards aspartate production. 

In [27]:
with model:
    overexpress = [model.reactions.HEX1, model.reactions.PPCK, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = overexpress, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_8 = model.optimize().objective_value
    max_yield_glucose = Asp_production_8 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_8)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 


             minimum    maximum
HEX1             0.0  30.814000
PPCK             0.0  23.314000
EX_asp__L_e      0.0   1.828549
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.596027557488559
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4596027557488559


In [28]:
with model:
    model.reactions.HEX1.lower_bound = 5
    model.reactions.PPCK.lower_bound = 5
    overexpress_lit= [model.reactions.HEX1, model.reactions.PPCK, model.reactions.EX_asp__L_e]
    fva = flux_variability_analysis(model, reaction_list = overexpress_lit, fraction_of_optimum=0.9)
    print(fva)
    model.objective = model.reactions.EX_asp__L_e
    Asp_production_9 = model.optimize().objective_value
    max_yield_glucose = Asp_production_9 / (-1*model.reactions.EX_glc__D_e.flux)
    print("Max. l-aspartate production [mmol gDW^-1 h^-1]:", Asp_production_9)
    print("Theoretical max. yield [mmol-asp / mmol-gluc]:", max_yield_glucose) 


             minimum    maximum
HEX1             5.0  31.439000
PPCK             5.0  27.689000
EX_asp__L_e      0.0   1.779529
Max. l-aspartate production [mmol gDW^-1 h^-1]: 4.129296669811868
Theoretical max. yield [mmol-asp / mmol-gluc]: 0.4129296669811868


As we can see the over expression of the genes seems to have a negative impact in terms of Aspartate production.Even tho a higher flux is conducted towards L-aspartae production, we know that it has a negative correlation with biomass accumulation. This correlation explaind the decrease in overal productivity observed in the suggeted over expression targets. When talking about the literature reported genes an additional factor must be considered, their over expression was coupled with several gene knock outs, meant to silence non favored products. The lower aspartate production may be due to those reactions staying intact and competing for substrate availability. Given the nature of our product over expression seems to not be ideal as an optimization strategy given that it would cause a considerable reduction in biomass formation. 

## References

Shimizu, K. (2013). Brief overview of metabolic regulation of a bacterial cell. In Elsevier eBooks (pp. 55–94). https://doi.org/10.1533/9781908818201.55

Piao, X., Wang, L., Lin, B., Chen, H., Liu, W., & Tao, Y. (2019). Metabolic engineering of Escherichia coli for production of L-aspartate and its derivative β-alanine with high stoichiometric yield. Metabolic Engineering, 54, 244–254. https://doi.org/10.1016/j.ymben.2019.04.012

Shibatani, T. (1996). Industrial application of immobilized biocatalysts in Japan. In Progress in Biotechnology (pp. 585–591). https://doi.org/10.1016/s0921-0423(96)80080-3
